In [1]:
import os
import numpy as np
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredPDFLoader,UnstructuredFileLoader,OnlinePDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

a="JDBN/t5-base-fr-qg-fquad"
b="iarfmoose/t5-base-question-generator"
c='t5-base'


model = T5ForConditionalGeneration.from_pretrained(a)
tokenizer = T5Tokenizer.from_pretrained(a)


text = "generate question: Ceci est un exemple de texte sur l'apprentissage automatique?. L'apprentissage automatique est un sous-ensemble de l'intelligence artificielle qui permet aux systèmes d'apprendre et de s'améliorer à partir de l'expérience sans être explicitement programmés. Il se concentre sur le développement d'algorithmes et de modèles statistiques pouvant être utilisés pour effectuer des tâches spécifiques. efficacement sans utiliser d'instructions en s'appuyant sur des modèles et des inférences."
# Encode the input text
input_ids = tokenizer.encode(text, return_tensors='pt')

# Generate questions using the model
output_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)

# Decode and print the generated questions
for output in output_ids:
    question = tokenizer.decode(output, skip_special_tokens=True)
    print(question)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qu'est-ce qui permet aux systèmes d'apprendre et de s'améliorer sans être explicitement programmés?


In [4]:
def question_generation_from_text(txt):
    txt="generate question: "+txt
    #print(txt)
    # Encode the input text
    input_ids = tokenizer.encode(txt, return_tensors='pt')
    # Generate questions using the model
    output_ids = model.generate(input_ids, max_length=500, num_beams=4, early_stopping=True)
    
    # Decode and print the generated questions
    for output in output_ids:
        question = tokenizer.decode(output, skip_special_tokens=True)
        return question

In [5]:
question_generation_from_text("la mosqué hassan 2 situé à casablanca")

'Où se trouve la mosquée hassan 2?'

In [19]:
question_generation_from_text("Coordinateur: Pr.XXX XXXX")

generate question: Coordinateur: Pr.XXX XXXX


'Quel est le nom du coordinateur?'

In [62]:
with open("txt_data/"+'LST__Génie_Informatique.txt','r',encoding="UTF-8") as f:
    print(question_generation_from_text(f.read()))

Qu'est-ce qu'un savoir-faire en informatique?


In [20]:
import os

def list_files_in_directory(directory_path):
    try:
        entries = os.listdir(directory_path)
        files = [entry for entry in entries if os.path.isfile(os.path.join(directory_path, entry))]
        return files
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


directory_path = "cours_pdf"
directory_path1 = "txt_data"
pdf_files = list_files_in_directory(directory_path)
txt_files = list_files_in_directory(directory_path1)

txt_files

['10ÈME_ÉDITION_DU_CONCOURS_FRANCOPHONE_INTERNATIONAL_MA_THÈSE_EN_180_SECONDES.txt',
 '14_eme_édition_du_Concours_national_de_linnovation_de_la_recherche-développement_et_de_la_technologie.txt',
 '1ER_CONGRÈS_INTERNATIONAL_RÉSILIENCE_ET_ADAPTATION_AUX_CATASTROPHES_NATURELLES.txt',
 '1_ère_édition_de_lE-Conférence_de_la_robotique_FST_Tanger-Afrique.txt',
 '2ND_INTERNATIONAL_CONGRESS_ON_COASTAL_RESEARCH_ICCR2.txt',
 '5ÈME_CONCLAVE_DE_SÉCURITÉ_AU_MAROC.txt',
 'ACTIVATION_DES_ADRESSES_E-MAILS_INSTITUTIONNELLES.txt',
 'Activation_des_emails_institutionnels_pour_les_étudiants_déjà_inscrits.txt',
 'Actualisation_des_Emplois_du_temps_de_la_session_de_Printemps_20202021_Travaux_dirigés.txt',
 'Actualisation_du_planning_des_examens_du_tronc_commun_du_cycle_licence___Session_dautomne_2020-2021.txt',
 'Affectations_PROVISOIRES_des_modules_des_semestres_S2_et_S4_du_cycle_licence__Session_de_printemps_20192020.txt',
 'Affectations_PROVISOIRES_des_modules_des_semestres_S2_et_S4_du_cycle_licence__Sess

In [6]:
question_generation_from_text("Le club Génie civil est un club scientifique de la faculté des sciences et techniques Tanger crée en 2014 par l’étudiant KALOUCH Hammam Qui vise a développer la culture scientifiques chez les étudiants de la FSTT ainsi que présenter la filière LST et MST Génie Civil de la FSTT et d’organiser plusieurs activité scientifiques dans le domaine de Génie civil, qui incluent des visites de chantiers, des conférences, des Ateliers et Un évènement annuel la journée Génie civil FSTT .")

"Qu'est-ce que le club Génie civil?"

In [28]:
with open("txt_data/"+'Club_Génie_civil.txt','r',encoding="UTF-8") as f:
    print(question_generation_from_text(f.read()))

Quel est le nom du club scientifique du Club Génie civil?


In [7]:
def questions_generation_from_text(txt):
    import re
    # Divide the text into segments based on sentence endings
    segments = re.split(r'[.!?][\s\n]+', txt)
    
    generated_questions = []
    
    for segment in segments:
        #print(segment)
        if segment.strip():  # Skip empty segments
            input_text = "generate question: " + segment.strip()
            
            # Encode the input text
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            
            # Generate questions using the model
            output_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
            
            # Decode and add the generated questions
            for output in output_ids:
                question = tokenizer.decode(output, skip_special_tokens=True)
                generated_questions.append(question)
    
    return generated_questions

In [9]:
with open("txt_data/"+'Formation_continue.txt','r',encoding="UTF-8") as f:
    print(questions_generation_from_text(f.read()))

["Quels sont les programmes de formation continue destinés aux professionnels qui souhaitent acquérir de nouvelles compétences ou approfondir leurs connaissances dans leur domaine d'activité?", "Qu'est-ce que l'informatique civile?"]


In [96]:
#pour générer des questions pertinantes pour chaque fichier de il fau cleaner le text et le segmenter:
# Commanceant par la collection des actualités
import pymongo
import os

# Connexion à MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  
db = client["DATA_FSTT"]  
collection = db["actualite_col"]

data=[]

# Traitement des documents
for document in collection.find():
    titre = document.get('titre', 'sans_titre')
    date = document.get('date', 'date_non_specifiee')
    lien = document.get('lien', 'lien_non_specifie')
    contenu = f"Titre: {titre.lower()}, Date: {date}. Pour plus de détails, veuillez consulter le lien suivant : {lien}"
    data.append(contenu)

#la collection des clubs
collection = db["club_col"]
# Traitement des documents
for document in collection.find():
    club = document.get('club', 'sans_nom')
    informations = document.get('informations', 'aucune_info_specifiee')
    contenu = f"Voici quelques information sur le Club {club}: {informations}."
    data.append(contenu)

collection = db["commission_col"]
for document in collection.find():
    membre = document.get('Membre', 'sans_nom')
    responsabilite = document.get('Responsabilité', 'responsabilité_non_specifiée')
    categorie = document.get('Categorie', 'catégorie_non_specifiée')
    commissions = document.get('Commission', [])

    # Création de la liste des commissions
    commissions_list = ", ".join(commissions)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = f"Mr/Me {membre} est le/la {responsabilite} de la faculté des sciences et techniques,il/elle est un/une membre dans les commissions suivantes : {commissions_list}. "
    data.append(contenu)

collection = db["departement_col"]
for document in collection.find():
    departement = document.get('Departement', 'sans_nom')
    chef = document.get('Chef', 'chef_non_specifié')
    email = document.get('Email', 'email_non_specifié')
    contenu = (f"Le département de {departement}, dirigé par {chef}, peut être contacté à l'adresse email {email}. ")
    data.append(contenu)


collection = db["presentation_col"]
for document in collection.find():
    titre = document.get('titre', 'sans_titre')
    texte = document.get('text', 'texte_non_specifié')
    contenu =f"{titre}: {texte}"
    data.append(contenu)



db = client["FORMATION_INITIAL"]  
collection = db["ci_col"]

# Fonction pour formater le programme en texte
def format_programme(programme):
    programme_text = ""
    for semestre, cours in programme.items():
        programme_text += f".{semestre}:"
        for index, cours_name in enumerate(cours):
            programme_text += f"  {index + 1}. {cours_name},"
    return programme_text

# Traitement des documents
for document in collection.find():
    filiere = document.get('filere', 'sans_nom')
    objectif = document.get('objectif', 'objectif_non_specifié')
    coordinateur = document.get('coordinateur', 'coordinateur_non_specifié')
    email = document.get('email', 'email_non_specifié')
    debouche = document.get('debouche', 'debouche_non_specifié')
    programme = document.get('programme', {})

    # Formatage du programme
    programme_text = format_programme(programme)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = (
        f"Filière: {filiere}\n\n"
        f"Objectif: {objectif}\n\n"
        f"Coordinateur: {coordinateur}\n"
        f"Email: {email}\n\n"
        f"Débouchés: {debouche}\n\n"
        f"Programme:{programme_text}"
    )


db = client["FORMATION_INITIAL"]  
collection = db["ci_col"]

# Fonction pour formater le programme en texte
def format_programme(programme):
    programme_text = ""
    for semestre, cours in programme.items():
        programme_text += f".{semestre}:"
        for index, cours_name in enumerate(cours):
            programme_text += f"{index + 1}. {cours_name},"
    return programme_text

# Traitement des documents
for document in collection.find():
    filiere = document.get('filere', 'sans_nom')
    objectif = document.get('objectif', 'objectif_non_specifié')
    coordinateur = document.get('coordinateur', 'coordinateur_non_specifié')
    email = document.get('email', 'email_non_specifié')
    debouche = document.get('debouche', 'debouche_non_specifié')
    programme = document.get('programme', {})

    # Formatage du programme
    programme_text = format_programme(programme)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = (
        f"La filière {filiere}, "
        f"à pour l objectif de {objectif}."
        f"Vous pouvez contacter le Coordinateur de filiere Mr./Me. {coordinateur} "
        f"Sur Email: {email} ."
        f"le débouchés de cette filiére sont comme suite;{debouche}."
        f"cette formation fournie un programme solide structuré comme suite:{programme_text}"
    )
    data.append(contenu)

collection = db["deust_col"]
for document in collection.find():
    filiere = document.get('filere', 'sans_nom')
    objectif = document.get('objectif', 'objectif_non_specifié')
    coordinateur = document.get('coordinateur', 'coordinateur_non_specifié')
    email = document.get('email', 'email_non_specifié')
    programme = document.get('programme', {})

    # Formatage du programme
    programme_text = format_programme(programme)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = (
        f"La filière {filiere}, "
        f"à pour l objectif de {objectif}."
        f"Vous pouvez contacter le Coordinateur de filiere Mr./Me. {coordinateur} "
        f"Sur Email: {email} ."
        f"cette formation fournie un programme solide structuré comme suite:{programme_text}"
    )
    data.append(contenu)

collection = db["licence_col"]
for document in collection.find():
    filiere = document.get('filere', 'sans_nom')
    objectif = document.get('objectif', 'objectif_non_specifié')
    coordinateur = document.get('coordinateur', 'coordinateur_non_specifié')
    email = document.get('email', 'email_non_specifié')
    debouche = document.get('debouche', 'debouche_non_specifié')
    programme = document.get('programme', {})

    # Formatage du programme
    programme_text = format_programme(programme)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = (
        f"La filière {filiere}, "
        f"à pour l objectif de {objectif}."
        f"Vous pouvez contacter le Coordinateur de filiere Mr./Me. {coordinateur} "
        f"Sur Email: {email} ."
        f"le débouchés de cette filiére sont comme suite;{debouche}."
        f"cette formation fournie un programme solide structuré comme suite:{programme_text}"
    )
    data.append(contenu)

collection = db["master_col"]
for document in collection.find():
    filiere = document.get('filere', 'sans_nom')
    objectif = document.get('objectif', 'objectif_non_specifié')
    coordinateur = document.get('coordinateur', 'coordinateur_non_specifié')
    email = document.get('email', 'email_non_specifié')
    debouche = document.get('debouche', 'debouche_non_specifié')
    programme = document.get('programme', {})

    # Formatage du programme
    programme_text = format_programme(programme)

    # Création du contenu du fichier sous forme de paragraphe
    contenu = (
        f"La filière {filiere}, "
        f"à pour l objectif de {objectif}."
        f"Vous pouvez contacter le Coordinateur de filiere Mr./Me. {coordinateur} "
        f"Sur Email: {email} ."
        f"le débouchés de cette filiére sont comme suite;{debouche}."
        f"cette formation fournie un programme solide structuré comme suite:{programme_text}"
    )
    data.append(contenu)



In [97]:
len(data)

406

In [98]:
#Creation de fichier CSV pour stocker les questions et les textes:
import csv
with open('data_QA.csv', 'w', newline='', encoding='UTF-8') as csvfile:
    fieldnames = ['text', 'question']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Generate questions for each item in the JSON data and write to CSV
    for item in data:
        question = question_generation_from_text(item)
        writer.writerow({'text': item, 'question': question})